In [1]:
cd "/content/drive/My Drive/Colab Notebooks/Dataworkshop_ML/MatrixDataworkshop/SignRoadsRecognition"

/content/drive/My Drive/Colab Notebooks/Dataworkshop_ML/MatrixDataworkshop/SignRoadsRecognition


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
%load_ext tensorboard

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

In [0]:
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(X_train,
             y_train,
             batch_size=params_fit.get('batch_size',128),
             epochs = params_fit.get('epochs', 5),
             verbose = params_fit.get('verbose', 1),
             validation_data = params_fit.get('validation_data', (X_train, y_train)),
             callbacks =[tensorboard_callback],
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test,scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return scoring(y_test_norm, y_pred)

def train_and_predict(model):
  model_trained = train_model(model,X_train,y_train)
  return predict(model_trained,X_test,y_test)

In [0]:
def get_cnn_v3(input_shape, num_classes):
 return Sequential([
      Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
      Conv2D(64,(3,3),activation='relu'),
      MaxPooling2D(),
      Dropout(0.5),
      Conv2D(128,(3,3),activation='relu',padding='same'),
      Conv2D(128,(3,3),activation='relu'),
      MaxPooling2D(),
      Dropout(0.3),
      Conv2D(256,(3,3),activation='relu',padding='same'),
      Conv2D(256,(3,3),activation='relu'),
      Dropout(0.3),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(512, activation='relu'),
      Dense(num_classes, activation='softmax'),
  ])

In [0]:
X_train_gray = color.rgb2gray(X_train).reshape(-1,32,32,1)
X_test_gray = color.rgb2gray(X_test).reshape(-1,32,32,1)

In [12]:
model = get_cnn_v3((32,32,1), num_classes)
model_trained = train_model(model, X_train_gray, y_train, params_fit={'epochs':15})
predict(model_trained,X_test_gray,y_test)

Epoch 1/15
272/272 [==============================] - 12s 45ms/step - loss: 2.7020 - accuracy: 0.2525 - val_loss: 0.4461 - val_accuracy: 0.8569
Epoch 2/15
272/272 [==============================] - 12s 43ms/step - loss: 0.2934 - accuracy: 0.9093 - val_loss: 0.0712 - val_accuracy: 0.9796
Epoch 3/15
272/272 [==============================] - 12s 43ms/step - loss: 0.1236 - accuracy: 0.9620 - val_loss: 0.0556 - val_accuracy: 0.9826
Epoch 4/15
272/272 [==============================] - 12s 43ms/step - loss: 0.0800 - accuracy: 0.9746 - val_loss: 0.0151 - val_accuracy: 0.9963
Epoch 5/15
272/272 [==============================] - 12s 43ms/step - loss: 0.0568 - accuracy: 0.9828 - val_loss: 0.0230 - val_accuracy: 0.9930
Epoch 6/15
272/272 [==============================] - 12s 43ms/step - loss: 0.0508 - accuracy: 0.9841 - val_loss: 0.0076 - val_accuracy: 0.9978
Epoch 7/15
272/272 [==============================] - 12s 43ms/step - loss: 0.0449 - accuracy: 0.9857 - val_loss: 0.0087 - val_accuracy:

0.9746031746031746

In [0]:
def get_model(params):
 return Sequential([
      Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
      Conv2D(64,(3,3),activation='relu'),
      MaxPooling2D(),

      Dropout(params['dropout_cnn_block_one']),
      Conv2D(128,(3,3),activation='relu',padding='same'),
      Conv2D(128,(3,3),activation='relu'),
      MaxPooling2D(),

      Dropout(params['dropout_cnn_block_two']),
      Conv2D(256,(3,3),activation='relu',padding='same'),
      Conv2D(256,(3,3),activation='relu'),
      Dropout(params['dropout_cnn_block_three']),
      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),
      Dense(512, activation='relu'),
      Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size',128)),
      epochs = 5,
      verbose = 0,
  )
  score = model.evaluate(X_test,y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuarcy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model':model}

In [15]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one', 0.3,0.5 ),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two', 0.3,0.5 ),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three', 0.3,0.5 ),
    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one', 0.3,0.7 ),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4694458814040075, 'dropout_cnn_block_three': 0.3891271125301852, 'dropout_cnn_block_two': 0.3184222968286529, 'dropout_dense_block_one': 0.6317980677939066}
accuarcy=0.9657596349716187
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.35755663759127293, 'dropout_cnn_block_three': 0.38169839174760184, 'dropout_cnn_block_two': 0.4355344203859298, 'dropout_dense_block_one': 0.32344224720500264}
accuarcy=0.960090696811676
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.35184556023963176, 'dropout_cnn_block_three': 0.4696604885951766, 'dropout_cnn_block_two': 0.42374659783289975, 'dropout_dense_block_one': 0.3642766066631294}
accuarcy=0.9546485543251038
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.42540356341043517, 'dropout_cnn_block_three': 0.46879161186785484, 'dropout_cnn_block_two': 0.3694371892263436, 'dropout_dense_block_one': 0.6124376115674117}
accuarcy=0.956462562084198
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.3105553966